### Calculate trajectory properties
#### List of tasks accomplished in this Jupyter Notebook:
- Add the size of each larvae into the trajectory CSV file
- Convert pixel locations to mm
- Zero out any off-screen values to the limits of the behavior arena
- Calculate instantaneous speed in mm per second
- Calculate instananeous angle (from horizontal, countercounterclockwise, in degrees)
- Calculate instantaneous change in heading (angle)
- Calculate the predicted concentration of chemical in the arena experienced by the larva at each time point
- Calculate the change in concentration relative to the previous timestep

In [1]:
import numpy as np
import pandas as pd
import scipy.interpolate
import os

In [2]:
def get_larva_size(animal, master_df):
    ''' Reference a master dataframe to find the size of each animal '''
    temp = master_df[master_df["animal_ID"] == animal]
    
    # Check that there is only one animal with this ID number
    assert len(temp) == 1
    mm = temp["larvae_length_mm"].values
    
    # Check that the size given by the master dataframe is a single number
    assert mm.size == 1
    return mm[0]

def hypotenuse(x1, y1, x0, y0):
    ''' Returns the length of the straight line vector between two points '''
    hyp = np.hypot(x1-x0, y1-y0)
    return hyp

def get_angle(x1, y1, x0, y0):
    ''' Calculate the angle from horizontal, counterclockwise '''
    angle = np.rad2deg(np.arctan2(y1-y0, x1-x0))
    return angle

def get_angle_delta(ang1, ang0):
    if (str(ang1) == 'nan') | (str(ang0) == 'nan'):
        return np.nan
    diff = ang1-ang0
    if diff >= 180:
        diff = -1 * (360-diff)
    elif diff <= -180:
        diff = 360 + diff
    return diff

def get_angle_delta_vector(angles1, angles0):
    diffs = []
    for ang1, ang0 in zip(angles1, angles0):
        diffs.append(get_angle_delta(ang1, ang0))
    return diffs

def get_bin(x, y):
    ''' Return the bin ID (rounded to nearest mm) '''
    # Bins are numbered starting at 0 so we lower to the number below:
    bin_x = min(79, np.floor(x))
    bin_y = min(29, np.floor(y))
    bin_val = bin_y*80 + bin_x
    bin_text = "bin_"+str(int(bin_val))
    return bin_text

def get_bin_vector(xs, ys):
    vals = []
    for x, y in zip(xs, ys):
        vals.append(get_bin(x, y))
    return vals

def get_concentration(bin_text, frame, ref_df):
    ''' Return the expected concentration experienced by the larva '''
    f = ref_df["frames"].values
    c = ref_df[bin_text].values
    interp = scipy.interpolate.interp1d(f, c, kind='linear')

    frame = min(max(f), frame)
    return interp(frame)

def get_concentration_vector(bin_texts, frames, ref_df):
    vals = []
    for bin_text, frame in zip(bin_texts, frames):
        vals.append(get_concentration(bin_text, frame, ref_df))
    return vals

In [3]:
master_df = pd.read_csv("./data/experiment_IDs/cleaned_static_data.csv")
master_df = master_df[master_df['dead'] != 'yes']
animals = master_df["animal_ID"].values

# Dataframe to use in finding the concentration values
ref_df = pd.read_csv("./data/fluorescein/bin_concentration_by_time_no_larvae.csv")

fps = 2 # frames per second in video
sh = 1 # direction in which to shift vector for delta calculations
walldist = 10 # maximum mm away from wall to count as "next to wall"

for animal in animals: 
    aID = animal[:9]
    pos = animal[10:]
    read = "./data/trajectories/video_csvs/"
    save = "./data/trajectories/video_calculations/"
    
    for val in ["A", "E"]:
        readname = read + aID + "-" + val + "-" + pos + ".csv"
        savename = save + aID + "-" + val + "-" + pos + ".csv"
            
        if not os.path.isfile(savename):
            try:
                df = pd.read_csv(readname)
                
                # Add the size of each larvae into the trajectory CSV file
                size = get_larva_size(animal, master_df)
                df['larvae_length_mm'] = size

                # Convert pixel locations to mm using known arena dimensions
                # (80 x 30 mm arena)
                df['pos_x_mm'] = 80*df['position_x']/df['pixel_width']
                df['pos_y_mm'] = 30*df['position_y']/df['pixel_height']

                # Zero out any off-screen values to the limits of the behavior arena
                df['pos_x_mm'] = df['pos_x_mm'].clip(lower=0, upper=80)
                df['pos_y_mm'] = df['pos_y_mm'].clip(lower=0, upper=30)

                # Calculate instantaneous speed in mm per second
                # Mltiply distance by fps to get speed per second
                df["speed_mm_s"] = hypotenuse(df['pos_x_mm'], df['pos_y_mm'], 
                                              df['pos_x_mm'].shift(sh), df['pos_y_mm'].shift(sh))*fps
                # Calculate speed in body lengths per second
                df["speed_BL"] = df['speed_mm_s']/size

                # Calculate instananeous angle (from horizontal, counterclockwise, in degrees)
                df["angle_counterclock"] = get_angle(df['pos_x_mm'], df['pos_y_mm'], 
                                               df['pos_x_mm'].shift(sh), df['pos_y_mm'].shift(sh))

                # Calculate instantaneous change in heading (angle)
                df["angle_delta"] = get_angle_delta_vector(df["angle_counterclock"], 
                                                    df["angle_counterclock"].shift(1))

                # Get the unique bin ID per time frame
                df["bin_ID"] = get_bin_vector(df["pos_x_mm"], df["pos_y_mm"])

                # Calculate the expected concentration
                df["concentration"] = get_concentration_vector(df["bin_ID"], df["frames"], ref_df)
                df["concentration_delta"] = df["concentration"] - df["concentration"].shift(1)

                # Calculate if larvae are moving
                df["moving"] = df["speed_mm_s"] >= 1

                # Calculate if larvae is executing a turn
                df["turn"] = abs(df["angle_delta"]) >= 30

                df.to_csv(savename, index=None)
            except:
                print(readname)
            
print("--- All files finished ---")

--- All files finished ---
